In [1]:
import numpy as np
from numpy import linalg as LA

import sys
import re
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf()
sc = SparkContext(conf=conf)
sc.setLogLevel('WARN')

loc = "./graph.txt"
#loc = sys.argv[1]


In [3]:
n = 1000

In [4]:
lines = sc.textFile(loc, 1)
line1 = lines.map(lambda l: re.split('\t',l))
line2 = line1.map(lambda x: (int(x[0]), int(x[1]))).distinct()
graph = line2.collect()

In [11]:
L = np.zeros((n, n), dtype = float)

for i in graph:
    L[i[0]-1, i[1]-1] = 1

In [12]:
Lt = L.transpose()

In [13]:
L

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.]])

# Using Spark

In [14]:
_L = sc.parallelize(L)
_Lt = sc.parallelize(Lt)

In [15]:
count = 0
_h = np.ones((n, 1), dtype = float)

while(count < 50):
    vec1 = _L.flatMap(lambda x: np.matmul(x, _h))
    a = vec1.collect()
    _a = np.asarray(a)
    _a = _a / np.max(_a)
    _a.shape = (1000, 1)
    vec2 = _Lt.flatMap(lambda x: np.matmul(x, _a))
    h = vec2.collect()
    _h = np.asarray(h)
    _h = _h / np.max(_h)
    _h.shape = (1000, 1)
    count += 1

# Postprocessing

In [16]:
h_result = list()
a_result = list()
for i in range(n):
    h_result.append((i+1, _h[i][0]))
    a_result.append((i+1, _a[i][0]))

In [17]:
sorted_h_result = sorted(h_result, key = lambda x: x[1], reverse = True)
sorted_a_result = sorted(a_result, key = lambda x: x[1], reverse = True)

In [18]:
for i in range(10):
    print(sorted_h_result[i][0], '\t', sorted_h_result[i][1])

893 	 1.0
16 	 0.9635572849634396
799 	 0.9510158161074016
146 	 0.9246703586198441
473 	 0.8998661973604047
624 	 0.8922197517765468
533 	 0.8832413304913613
780 	 0.8800357843384586
494 	 0.8749884615072088
130 	 0.8465465351844075


In [19]:
for i in range(10):
    print(sorted_a_result[i][0], '\t', sorted_a_result[i][1])

840 	 1.0
155 	 0.9499618624906542
234 	 0.8986645288972261
389 	 0.8634171101843788
472 	 0.8632841092495215
444 	 0.8229716669865107
666 	 0.8007139982829948
499 	 0.7966145570824411
737 	 0.7746877622644929
137 	 0.7715148677313686
